In [5]:
import mlflow
logged_model = 'runs:/a2ceb29ed2b841a199135321a63b4464/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

2022/12/09 22:10:04 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - torch-geometric (current: 2.2.0, required: torch-geometric==2.0.4)
 - torch-scatter (current: 2.1.0+pt113cpu, required: torch-scatter==2.0.9)
 - torch-sparse (current: 0.6.15+pt113cpu, required: torch-sparse==0.6.14)
 - torch (current: 1.13.0, required: torch==1.11.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


NameError: name 'data' is not defined

In [ ]:
import torch, torch_geometric
import pandas as pd

unexpected_data = []
unexpected_tie = 0
unexpected_win = 0

unlikely_data = []
unlikely_tie = 0
unlikely_win = 0
for _ in range(4000):
    vis = torch.load(f'data/processed_unnormalized/data_{_}.pt')
    print("processing: ", _)
    a = vis.x[:11]
    b = vis.x[11:]
    A = a.mean(axis=0)
    B = b.mean(axis=0)
    count = 0
    for p, q in zip(A, B):
        if(p>q):
            count+=1

    # 客隊為q, 0
    if(count >= 30 or count <= 8): # 主隊贏客隊超過30個能力 or 只贏小於8個(差距80%)
        if(vis.y == 1): # 雙方平手
            unexpected_tie += 1
            unexpected_data.append([A, B, count, vis.y])
        if(count >= 30 and vis.y == 0): # 主隊贏客隊超過30個能力但客隊獲勝
            unexpected_win += 1
            unexpected_data.append([A, B, count, vis.y])
        if(count <= 8 and vis.y == 2): # 主隊只贏小於8個能力但獲勝
            unexpected_win += 1
            unexpected_data.append([A, B, count, vis.y])
        
    if(count >= 25 or count <= 13): # 主隊贏客隊超過25個能力 or 只贏小於13個(贏66%的項目)
        if(vis.y == 1): # 雙方平手
            unlikely_tie += 1
            unlikely_data.append([A, B, count, vis.y])
        if(count >= 25 and vis.y == 0): # 主隊贏客隊超過25個能力但客隊獲勝
            unlikely_win += 1
            unlikely_data.append([A, B, count, vis.y])
        if(count <= 13 and vis.y == 2): # 主隊只贏小於13個能力但獲勝
            unlikely_win += 1
            unlikely_data.append([A, B, count, vis.y])
        
unexpected_data = pd.DataFrame(unexpected_data, columns = column_names)
unexpected_data.to_csv('unexpected_data.csv')
unlikely_data = pd.DataFrame(unlikely_data, columns = column_names)
unlikely_data.to_csv('unlikely_data.csv')

print("unexpected_tie: ", unexpected_tie)
print("unexpected_win: ", unexpected_win)
print("unlikely_tie: ", unlikely_tie)
print("unlikely_win: ", unlikely_win)